## Data Mining and Machine Learning
### Logistic Regression: The  ROC curve
### Libraries:scikit-learn and h2o
#### Edgar Acuna

#### November 2021
####  Dataset: Diabetes

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import h2o
from h2o.estimators.glm  import H2OGeneralizedLinearEstimator
import warnings
warnings.filterwarnings('ignore')
#h2o.connect()
#h2o.no_progress()
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,2 hours 44 mins
H2O_cluster_timezone:,America/Halifax
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_eacun_txweyd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,75.8 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### I Regresion Logistica para Diabetes usando scikit learn

In [3]:
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
#La variable de respuesta y debe ser binaria (0,1)
y=data['class']-1
X=data.iloc[:,0:8]
#Haciendo la regresion logistica ya calculando su precision
model = LogisticRegression()
model = model.fit(X, y)
print(model.coef_)

[[ 1.17252324e-01  3.35996755e-02 -1.40874200e-02 -1.27053477e-03
  -1.24031220e-03  7.72025216e-02  1.41904116e+00  1.00355163e-02]]


In [4]:
# Tasa de precision
print(model.score(X, y))

0.7825520833333334


In [5]:
predictions = model.predict(X)
print(predictions)

[1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1
 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0
 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 0 0 1 1
 0 1 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1
 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 1 0 1 0 0 0 

In [6]:
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       500
           1       0.74      0.57      0.65       268

    accuracy                           0.78       768
   macro avg       0.77      0.73      0.75       768
weighted avg       0.78      0.78      0.77       768



In [8]:
#Estimacion de la precision con k=3 vecinos  por el metodo  "holdout 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=0)
X_train, y_train

X_test, y_test

model = LogisticRegression()
model = model.fit(X_train, y_train)
# Tasa de precision
print(model.score(X_test, y_test))

0.7792207792207793


In [9]:
prob3=pd.DataFrame(model.predict_proba(X_test))
a=prob3.max(axis=1)
print('Probability of classification',(a[a>.90].shape[0])/prob3.shape[0])

Probability of classification 0.22943722943722944


### II. ROC curve using scikit-learn

In [ ]:
#Hallando las probabilidades posteriores
probs = model.predict_proba(X)
preds = probs[:,1]
false_positive_rate, true_positive_rate, thresholds = roc_curve(y, preds)
roc_auc = auc(false_positive_rate, true_positive_rate)

In [ ]:
plt.title('The ROC curve')
plt.plot(false_positive_rate, true_positive_rate, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1.0])
plt.ylim([0,1.0])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

The AUC value represents the area under the curve ROC (azul). Ia classifier has an  AUC between .9 and 1 then its predictions are very good, if the AUC lies between  .8 y .89 its prediction are good. A poor classifier is one with an  AUC less than de .60 de AUC.


### III Intersection of the sensitivity and  specifity curves to choose the threshold

In [ ]:
plt.title('Choice of the optimal Threshold')
plt.plot(thresholds, true_positive_rate, 'b',label='Sensitivity')
plt.legend(loc='lower right')
plt.plot(thresholds, 1-false_positive_rate,'r--')
plt.xlim([0,1.0])
plt.ylim([0,1.0])
plt.ylabel('Sensitivity ')
plt.xlabel('Probability')
plt.show()

El threshold que deberia ser usado en lugar de p=.5 para hacer la clasificacion sera aprox .35

### IV.  ROC curve using H20

In [ ]:
diabetes = h2o.import_file("https://academic.uprm.edu/eacuna/diabetes.dat")
myx=['C1','C2','C3','C4','C5','C6','C7','C8']
diabetes['C9']=diabetes['C9'].asfactor()
myy='C9'
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0, compute_p_values = True)
glm_model.train(myx, myy, training_frame= diabetes)
glm_model
glm_model._model_json['output']['coefficients_table']

In [ ]:
perf = glm_model.model_performance()  #train=True is the default, so it's not needed
perf.plot()

In [ ]:
#Effect after using the threshokd
#Number of instances assigned to class 1 using p=.5
dp1=data[preds>.5]
dp1['class'].value_counts()

In [ ]:
pred1=[1]*768
pred1=np.array(pred1)
pred1[dp1.index]=2
pd.crosstab(pred1,data['class'],rownames=['class_pred'])

In [ ]:
#accuracy
595/768

In [ ]:
#Number of instances assigned to class 1 using p=.35
dp2=data[preds>.35]
dp2['class'].value_counts()

In [ ]:
pred1=[1]*768
pred1=np.array(pred1)
pred1[dp2.index]=2
pd.crosstab(pred1,data['class'],rownames=['class_pred'])

In [ ]:
#New accuracy
591/768